In [ ]:
# implement you experimentation code....

## Import the required libraries

In [3]:
import kfp
import kfp.dsl as dsl
from datetime import datetime
import kfp.compiler as compiler

## Load the pipeline components

In [6]:
component_store = kfp.components.ComponentStore(
  local_search_paths=['../components'])

## Create pipeline ops

In [7]:
add_op = component_store.load_component('my_add')
divide_op = component_store.load_component('my_divide')

## Define pipeline

In [8]:
def pipeline(
  x_value: int=1,
  y_value: int=1,
  z_value: int=1,
):

  add_step = add_op(x_value=x_value, y_value=y_value)
  add_step.set_display_name('Add x and y')
  add_result = add_step.outputs
  sum_value = add_result['sum']
  is_even = sum_value != 0
  with kfp.dsl.Condition(is_even):
    divide_step = divide_op(x_value=sum_value, y_value=z_value)
    divide_step.set_display_name('Divide sum by z')
    add_step2 = add_op(
      x_value=divide_step.outputs['quotient'],
      y_value=divide_step.outputs['remainder'])
    add_step2.set_display_name('Add quotient and remainder')

## Compile pipeline

In [17]:
compiler.Compiler().compile(
    pipeline, 'pipeline.tar.gz')

## Up load pipeline

In [23]:
pipeline_name = "CalQUBO_mock"
client.pipeline_uploads.upload_pipeline(
    uploadfile = "pipeline.tar.gz", description = "description", name = pipeline_name, async_req=True
)

## Run pipeline

In [21]:
args = {
    'x_value': 4,
    'y_value': 1,
    'z_value': 8,
    
}
client = kfp.Client(namespace='default', host='6b415e369a40e0ab-dot-asia-east1.pipelines.googleusercontent.com')
run_id = 'run-' + datetime.now().strftime('%Y%m%d-%H%M%S')
experiment = client.create_experiment(name='exp-dev')
pipeline_package = 'pipeline.tar.gz'

client.run_pipeline(
    experiment.id,
    job_name=run_id,
    pipeline_package_path=pipeline_package,
    params=args
)

{'created_at': datetime.datetime(2020, 11, 13, 10, 39, 45, tzinfo=tzutc()),
 'description': None,
 'error': None,
 'finished_at': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzutc()),
 'id': 'e6e2babd-f3cc-48f3-8417-2a89a455660e',
 'metrics': None,
 'name': 'run-20201113-193945',
 'pipeline_spec': {'parameters': [{'name': 'x_value', 'value': '4'},
                                  {'name': 'y_value', 'value': '1'},
                                  {'name': 'z_value', 'value': '8'}],
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'pipeline_name': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "metadata": '
                                        '{"generateName": "pipeline-", '
                                        '"annotations": '
                                        '{"pipelines.